In [1]:
import warnings
import numpy as np
import pandas as pd
from datetime import datetime, timedelta

warnings.filterwarnings("ignore", category=UserWarning)

In [2]:
# Reading short imputed master dataframe
master_df = pd.read_csv("data/2016_2021_master_df_short_imputed.csv")
master_df["DATE_PST"] = pd.to_datetime(master_df["DATE_PST"])
master_df.set_index("DATE_PST", inplace=True)
display(master_df.head())
print(master_df.shape)

,STATION,PM 2.5,MISSING,MISSING_SEQ,MISSING_SAMPLE
DATE_PST,,,,,
2016-01-01 01:00:00,Vancouver_Clark_Drive_PM25,30.557,0,0,0
2016-01-01 02:00:00,Vancouver_Clark_Drive_PM25,34.661,0,0,0
2016-01-01 03:00:00,Vancouver_Clark_Drive_PM25,35.419,0,0,0
2016-01-01 04:00:00,Vancouver_Clark_Drive_PM25,24.335,0,0,0
2016-01-01 05:00:00,Vancouver_Clark_Drive_PM25,29.336,0,0,0


(228056, 5)


In [3]:
# Creating independent dataframes
datasets = {}
for station in master_df["STATION"].unique():
    missing = master_df["PM 2.5"][master_df["STATION"] == station].isna().sum()
    # Missing values
    print(f"Missing values at {station}: {missing}")

    # New dataframes
    datasets[station[:-5]] = master_df[master_df["STATION"] == station].drop(columns="STATION")

Missing values at Vancouver_Clark_Drive_PM25: 1298
Missing values at Vancouver_International_Airport_#2_PM25: 342
Missing values at North_Vancouver_Mahon_Park_PM25: 447
Missing values at North_Vancouver_Second_Narrows_PM25: 2088


In [4]:
for dataset in datasets:
    print(f"{dataset} dimensions: {datasets[dataset].shape}")

Vancouver_Clark_Drive dimensions: (57014, 4)
Vancouver_International_Airport_#2 dimensions: (57014, 4)
North_Vancouver_Mahon_Park dimensions: (57014, 4)
North_Vancouver_Second_Narrows dimensions: (57014, 4)


In [5]:
# Imputation on 2016 data
for station in datasets:
    # Linear interpolating to fill in missing data on first 60 days
    first60subset = datasets[station][datetime(2016, 1, 1, 1):datetime(2016, 3, 1)]
    print(f"Missing values for {station} on first 60 days: {first60subset['PM 2.5'].isna().sum()}")
    first60subset.interpolate(inplace=True)
    datasets[station].loc[first60subset.index, "PM 2.5"] = first60subset["PM 2.5"]

Missing values for Vancouver_Clark_Drive on first 60 days: 29
Missing values for Vancouver_International_Airport_#2 on first 60 days: 0
Missing values for North_Vancouver_Mahon_Park on first 60 days: 0
Missing values for North_Vancouver_Second_Narrows on first 60 days: 0


/tmp/ipykernel_1436/4270651446.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  first60subset.interpolate(inplace=True)


In [6]:
for station in datasets:
    missing = datasets[station]["PM 2.5"].isna().sum()
    print(f"Missing values at {station}: {missing}")

Missing values at Vancouver_Clark_Drive: 1269
Missing values at Vancouver_International_Airport_#2: 342
Missing values at North_Vancouver_Mahon_Park: 447
Missing values at North_Vancouver_Second_Narrows: 2088


In [11]:
from darts import TimeSeries
from darts.metrics import mae, rmse
from darts.models import RNNModel
from darts.utils.timeseries_generation import datetime_attribute_timeseries
from darts.dataprocessing.transformers import Scaler
from pytorch_lightning.callbacks.early_stopping import EarlyStopping

results_dict = {}

# LSTM imputation
first = True
for station in datasets:
    if first:
        minimum_date = datetime(2016, 3, 1)
        maximum_date = datetime(2017, 1, 15)
        start_date = minimum_date - timedelta(45)
        split_date = minimum_date
        end_date = minimum_date + timedelta(15)
        first = False
    
    while split_date < maximum_date:
        subset = datasets[station].loc[start_date:end_date, "PM 2.5"]
        series = TimeSeries.from_dataframe(subset.to_frame())

        # Training and validation split
        train, val = series.split_before(pd.Timestamp(split_date))

        ## Creating month, day-of-week, and hour covariates
        # Month covariates
        month_series = datetime_attribute_timeseries(
            series, attribute="month", one_hot=True
        )

        # Weekday covariates
        weekday_series = datetime_attribute_timeseries(
            series, attribute="weekday", one_hot=True
        )

        # Hour covariates
        hour_series = datetime_attribute_timeseries(
            series, attribute="hour", one_hot=False
        )
        # Scaling hour covariates
        hour_series = Scaler().fit_transform(hour_series)

        # Stacking covariates
        covariates = month_series.stack(weekday_series)
        covariates = covariates.stack(hour_series)

        # Model name
        model_name = f"LSTM_{station}_split-{split_date.strftime('%Y-%m-%d')}"
        results_dict[model_name] = {}

        # Early stopping
        stopper = EarlyStopping(
            monitor="val_loss",
            patience=10,
            min_delta=0.02,
            mode="min"
        )

        # LSTM model
        my_model = RNNModel(
            model="LSTM",
            hidden_dim=200,
            dropout=0.1,
            batch_size=32,
            n_epochs=100,
            optimizer_kwargs={"lr": 5e-4},
            model_name=model_name,
            log_tensorboard=True,
            random_state=123,
            training_length=20,
            input_chunk_length=21,
            force_reset=True,
            save_checkpoints=True,
            pl_trainer_kwargs={
                "callbacks": [stopper],
                "accelerator": "gpu",
                "devices": -1,
                "auto_select_gpus": True
            }
        )

        # Training
        my_model.fit(
            train,
            future_covariates=covariates,
            val_series=val,
            val_future_covariates=covariates,
            verbose=True,
        )

        # Validation
        pred_val = my_model.predict(n=len(val), future_covariates=covariates)
        my_model.load_from_checkpoint(model_name=model_name, best=True)

        # Storing results
        results_dict[model_name]["best_model"] = my_model
        results_dict[model_name]["covariates"] = covariates
        results_dict[model_name]["train"] = train
        results_dict[model_name]["val"] = val
        results_dict[model_name]["pred_val"] = pred_val
        results_dict[model_name]["MAE"] = mae(pred_val, val)
        results_dict[model_name]["RMSE"] = rmse(pred_val, val)

        start_date = start_date + timedelta(15)
        split_date = split_date + timedelta(15)
        end_date = end_date + timedelta(15)

/home/erich/projects/Air-We-Breath/.awb/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2022-12-05 22:53:07 pytorch_lightning.trainer.connectors.accelerator_connector INFO: Auto select gpus: [0]
2022-12-05 22:53:08 pytorch_lightning.utilities.rank_zero INFO: GPU available: True (cuda), used: True
2022-12-05 22:53:08 pytorch_lightning.utilities.rank_zero INFO: TPU available: False, using: 0 TPU cores
2022-12-05 22:53:08 pytorch_lightning.utilities.rank_zero INFO: IPU available: False, using: 0 IPUs
2022-12-05 22:53:08 pytorch_lightning.utilities.rank_zero INFO: HPU available: False, using: 0 HPUs
2022-12-05 22:53:13 pytorch_lightning.accelerators.cuda INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
2022-12-05 22:53:13 pytorch_lightning.callbacks.model_summary INFO: 
  | Name          | Type             | 

Epoch 25: 100%|██████████| 45/45 [00:01<00:00, 38.46it/s, loss=7.86, v_num=logs, train_loss=4.760, val_loss=7.500]


2022-12-05 22:53:46 pytorch_lightning.accelerators.cuda INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting DataLoader 0: 100%|██████████| 1/1 [00:00<00:00,  5.40it/s]


2022-12-05 22:53:46 pytorch_lightning.trainer.connectors.accelerator_connector INFO: Auto select gpus: [0]
2022-12-05 22:53:46 pytorch_lightning.utilities.rank_zero INFO: GPU available: True (cuda), used: True
2022-12-05 22:53:46 pytorch_lightning.utilities.rank_zero INFO: TPU available: False, using: 0 TPU cores
2022-12-05 22:53:46 pytorch_lightning.utilities.rank_zero INFO: IPU available: False, using: 0 IPUs
2022-12-05 22:53:46 pytorch_lightning.utilities.rank_zero INFO: HPU available: False, using: 0 HPUs
2022-12-05 22:53:47 pytorch_lightning.accelerators.cuda INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
2022-12-05 22:53:47 pytorch_lightning.callbacks.model_summary INFO: 
  | Name          | Type             | Params
---------------------------------------------------
0 | criterion     | MSELoss          | 0     
1 | train_metrics | MetricCollection | 0     
2 | val_metrics   | MetricCollection | 0     
3 | rnn           | LSTM             | 178 K 
4 | V             | Linear    

...

/home/erich/projects/Air-We-Breath/.awb/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/erich/projects/Air-We-Breath/.awb/lib/python3.9/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/erich/projects/Air-We-Breath/.awb/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/erich/projects/Air-We-Breath/.awb/lib/python3.9/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
2022-12-05 22:56:28 pytorch_lightning.trainer.connectors.accelerator_connector INFO: Auto select gpus: [0]
2022-12-05 22:56:29 pytorch_lightning.utilities.rank_zero INFO: GPU available: True (cuda), used: True
2022-12-05 22:56:29 pytorch_

Epoch 0: 100%|██████████| 45/45 [00:01<00:00, 34.18it/s, loss=nan, v_num=logs, train_loss=nan.0, val_loss=nan.0]


2022-12-05 22:56:30 pytorch_lightning.accelerators.cuda INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting DataLoader 0: 100%|██████████| 1/1 [00:00<00:00,  6.25it/s]


/home/erich/projects/Air-We-Breath/.awb/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/erich/projects/Air-We-Breath/.awb/lib/python3.9/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/erich/projects/Air-We-Breath/.awb/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/erich/projects/Air-We-Breath/.awb/lib/python3.9/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
